In [1]:
import json
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
import itertools
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sys, os
import traceback
from itertools import chain
import numpy as np
import pandas as pd

import progressbar

import tensorflow as tf
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
def read_data(rootdir):
    data_list = []
    for subdir, dirs, files in os.walk(rootdir):
        for fname in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + fname
            with open(filepath) as f:
                for line1,line2 in itertools.izip_longest(*[f]*2):
                    try:
                        text = ast.literal_eval(line1)
                        label = ast.literal_eval(line2)
                        
#                         for i in range(len(label)):
#                             if label[i] == 'NA':
#                                 label[i] = 'O'
                        n_labels = set(label)
                        if len(text) > 3 and len(n_labels) > 1:
                            sentence = zip(text, label)
                            sentence.append(('.', 'O'))
                            sentence = [item for item in sentence if item[0].strip() != '']
                            data_list.append(sentence)
                    except:
                        pass
            f.close()
    return data_list

In [4]:
data_list = read_data('../data/input/output_chebi/')

print data_list[0]

# data_list = data_list[:3000]

# words = list(chain.from_iterable(data_list))
# print words

print len(data_list)

words = list(set(y[0] for x in data_list for y in x))
words.append("ENDPAD")
n_words = len(words)
print n_words

tags = list(set(y[1] for x in data_list for y in x))
n_tags = len(tags)
print n_tags

[('The', 'O'), ('functional', 'O'), ('diversity', 'O'), ('of', 'O'), ('the', 'O'), ('novel', 'O'), ('conespecific', 'O'), ('genes', 'O'), ('identified', 'O'), ('here', 'O'), ('indicates', 'O'), ('molecular', 'CHEBI:23367'), ('differences', 'O'), ('between', 'O'), ('rods', 'O'), ('and', 'O'), ('cones', 'O'), ('extending', 'O'), ('far', 'O'), ('beyond', 'O'), ('those', 'O'), ('previously', 'O'), ('discovered', 'O'), ('.', 'O')]
3127
11109
309


In [5]:
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# le.fit(list(y[1] for x in data_list for y in x))

# labels2idx = dict(zip(le.classes_, le.transform(le.classes_)))

# idx2la = {labels2idx[k]:k for k in labels2idx}

In [6]:
max_len = 75
max_len_char = 10

word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["O"] = 0
idx2word = {i: w for w, i in word2idx.items()}
tag2idx = {t: i + 1 for i, t in enumerate(tags)}
tag2idx["O"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}

# print word2idx['Hybrid']
# print tag2idx["O"]

In [7]:
print tag2idx

{'CHEBI:32700': 1, 'CHEBI:27897': 2, 'CHEBI:16223': 3, 'CHEBI:28748': 290, 'CHEBI:23114': 6, 'CHEBI:17158': 7, 'CHEBI:32460': 8, 'CHEBI:39442': 9, 'CHEBI:26130': 10, 'CHEBI:49637': 12, 'CHEBI:22908': 13, 'CHEBI:15377': 139, 'CHEBI:25741': 14, 'CHEBI:26333': 15, 'CHEBI:32535': 266, 'CHEBI:16827': 16, 'CHEBI:22586': 17, 'CHEBI:36420': 98, 'CHEBI:22582': 72, 'CHEBI:15379': 19, 'CHEBI:34683': 20, 'CHEBI:26708': 21, 'CHEBI:22984': 22, 'CHEBI:16412': 23, 'CHEBI:24753': 24, 'CHEBI:6636': 25, 'CHEBI:23614': 26, 'CHEBI:16381': 27, 'CHEBI:23995': 28, 'CHEBI:16150': 29, 'CHEBI:24009': 213, 'CHEBI:15355': 274, 'CHEBI:32145': 33, 'CHEBI:37407': 106, 'CHEBI:16234': 35, 'CHEBI:17126': 36, 'CHEBI:16236': 37, 'CHEBI:42645': 38, 'CHEBI:24669': 291, 'CHEBI:29987': 40, 'CHEBI:22359': 41, 'CHEBI:27214': 42, 'CHEBI:41774': 43, 'CHEBI:17246': 108, 'CHEBI:22599': 45, 'CHEBI:42266': 46, 'CHEBI:30956': 47, 'CHEBI:15738': 48, 'CHEBI:28300': 49, 'CHEBI:32630': 245, 'CHEBI:22918': 50, 'CHEBI:16933': 32, 'CHEBI:153

In [8]:
from keras.preprocessing.sequence import pad_sequences
X_word = [[word2idx[w[0]] for w in s] for s in data_list]

In [9]:
X_word = pad_sequences(maxlen=max_len, sequences=X_word, value=word2idx["O"], padding='post', truncating='post')

In [10]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)

98


In [11]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["O"] = 0

In [12]:
X_char = []
for sentence in data_list:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))


In [13]:
y = [[tag2idx[w[1]] for w in s] for s in data_list]

In [14]:
y = pad_sequences(maxlen=max_len, sequences=y,value=tag2idx["O"], padding='post', truncating='post')

In [15]:
from sklearn.model_selection import train_test_split
X_word_tr, X_word_te, y_tr, y_te = train_test_split(X_word, y, test_size=0.3, random_state=2018)
X_char_tr, X_char_te, _, _ = train_test_split(X_char, y, test_size=0.3, random_state=2018)

In [16]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D

# input and embedding for words
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=n_words + 2, output_dim=100,
                     input_length=max_len, mask_zero=True)(word_in)

# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=100,
                           input_length=max_len_char, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(GRU(units=100, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(GRU(units=50, return_sequences=True,
                               recurrent_dropout=0.6))(x)
out = TimeDistributed(Dense(n_tags + 1, activation="sigmoid"))(main_lstm)

model = Model([word_in, char_in], out)

In [17]:
from keras.optimizers import RMSprop

rmsopt = RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
model.compile(optimizer=rmsopt, loss="sparse_categorical_crossentropy", metrics=["acc"])

In [ ]:
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 75, 10)       0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 75)           0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 75, 10, 100)  10000       input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 75, 100)      1111100     input_1[0][0]                    
__________________________________________________________________________________________________
time_distr

In [ ]:
history = model.fit([X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    np.array(y_tr).reshape(len(y_tr), max_len, 1),
                    batch_size=32, epochs=15, validation_data=([X_word_te,
                     np.array(X_char_te).reshape((len(X_char_te), max_len, max_len_char))],
                    np.array(y_te).reshape(len(y_te), max_len, 1)),
                    verbose=1)

Train on 2188 samples, validate on 939 samples
Epoch 1/15
2188/2188 [==============================] - 21s 9ms/step - loss: 0.5779 - acc: 0.9367 - val_loss: 0.1960 - val_acc: 0.9672
Epoch 2/15
2188/2188 [==============================] - 19s 9ms/step - loss: 0.1495 - acc: 0.9742 - val_loss: 0.1120 - val_acc: 0.9817
Epoch 3/15
 128/2188 [>.............................] - ETA: 15s - loss: 0.0875 - acc: 0.9807

In [ ]:
hist = pd.DataFrame(history.history)
hist.to_csv('../data/results/performance/Chebi_CHAR_GRU_Based.tsv', sep='\t')

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["loss"], label='Training Loss')
plt.plot(hist["val_loss"], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["acc"], label='Training Accuracy')
plt.plot(hist["val_acc"], label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report

predicted = model.predict([X_word_te,
                        np.array(X_char_te).reshape((len(X_char_te),
                                                     max_len, max_len_char))])
true = []
predicted_val = []
word = []
for i in range(len(predicted)):
    p_tmp = np.argmax(predicted[i], axis=-1)
    t_tmp = y_te[i]
    word.append(X_word_te[i])
    predicted_val.append(p_tmp)
    true.append(t_tmp)

print word[2]
print predicted_val[2]
print true[2]

wd = [list(map(lambda x: idx2word[x], y)) for y in word]    
p = [list(map(lambda x: idx2tag[x], y)) for y in predicted_val]
t = [list(map(lambda x: idx2tag[x], y)) for y in true]


print wd[2]
print p[2]
print t[2]

report = classification_report(list(itertools.chain.from_iterable(t)), 
                                    list(itertools.chain.from_iterable(p)))
print "\n"
print report

In [ ]:
i = 300
pred = np.argmax(predicted[i], axis=-1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, tr, pred in zip(X_word_te[i], y_te[i], pred):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w], idx2tag[tr], idx2tag[pred]))

# pd = model.predict(np.array([X_te[i]]))
# pd = np.argmax(pd, axis=-1)
# true = np.argmax(y_te[i], -1)
# print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
# print(30 * "=")

# for w, t1, pred in zip(X_te[i], true, pd[0]):
#     if w != 0:
#         print("{:15}: {:5} {}".format(words[w-1], tags[t1], tags[pred]))

In [ ]:
import pandas as pd

df = pd.DataFrame(
{
    'Predicted' : list(itertools.chain.from_iterable(p)),
    'Ground Truth': list(itertools.chain.from_iterable(t)),
    'Word' : list(itertools.chain.from_iterable(wd)),
}
)


In [ ]:
df = df[df['Word'] != 'O']

In [ ]:
df.to_csv('../data/results/Chebi_CHAR_GRU_Based.tsv', sep='\t')

In [ ]:
%run ./accuracy/src/computeSim.py ../data/results/Chebi_CHAR_GRU_Based.tsv ../data/validation_data/Chebi_AllSubsumers.tsv